In [1]:
import requests
import re
import pymongo

# Mongo db connect
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
dashboard = mongo_client["BestmomDatascraping"]
cl_data = dashboard['ScrapingFromList']

headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    }

url = 'https://www.1mg.com/drugs-therapeutic-classes/drug-class-142'

url_page = requests.get(url, headers = headers, timeout = 60).text


for i in url_page.split('href=')[1:]:
    source_link = re.search('"(.*?)"',i)
    if source_link:
        source_link = source_link.group(1)
    if '/drugs/' not in source_link:
        continue
    if 'http' not in source_link:
        source_link = 'https://www.1mg.com' + source_link
  
    page = requests.get(source_link, headers = headers, timeout = 60).text
    
    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        continue 

    Active_Drug = re.search('class="saltInfo DrugHeader__meta-value___vqYM0">.*?>(.*?)<',page)
    if Active_Drug:
        Active_Drug = Active_Drug.group(1)

    Synonyms = re.search('class="saltInfo DrugHeader__underline___f4HNk DrugHeader__meta-value___vqYM0">.*?>(.*?)<',page)
    if Synonyms:
        Synonyms = Synonyms.group(1)
    
    Brand = re.search('class="style__font-bold___1k9Dl style__font-14px___YZZrf style__flex-row___2AKyf style__space-between___2mbvn style__padding-bottom-5px___2NrDR">(.*?)</',i)
    if Brand:
        Brand = Brand.group(1)
    Brand = re.sub('<.*?>','',Brand,re.S)
    
    Dosage_form = str(Brand).split()
    for i in Dosage_form[-1:]:
        Dosage_form = i
        
    side_effect = re.search('class="DrugOverview__list-container___2eAr6 DrugOverview__content___22ZBX">(.*?)</ul>',page)
    if side_effect:
        side_effect = side_effect.group(1)
    side_effect = re.sub('<.*?>',' ',side_effect,re.S)

    Pricing = re.search('class="SubstituteList__top-row___1tAmD">.*?<div>.*?<div>(.*?)<',page)
    if Pricing:
        Pricing = Pricing.group(1)

    Alternate_Brand = ''
    Alternate_brand_find = re.findall('<div\s*class="SubstituteItem__name___PH8Al">(.*?)<div\s*class="SubstituteItem__save-text___1DPP8">',page)
    
    for i in Alternate_brand_find:
        i = re.sub('<.*?>',' ',i,re.S)

        comp = re.search('Tablet(.*?)₹',i)
        if not comp:
            continue
        if comp:
            comp = comp.group(1)
        if comp in i:
            i = re.sub(f'{comp}',' :--',i,re.S)

        Alternate_Brand += f'{i}, \n'

    # storing the above data in a dictionary
    clientdata ={
                    "source_link":source_link,
                    "Active_Drug":Active_Drug,
                    "Synonyms":Synonyms,
                    "Brand": Brand,
                    "Dosage_form":Dosage_form,
                    "side_effect":side_effect,
                    "Pricing":Pricing,
                    "Alternate_Brand":Alternate_Brand
                } 

    cl_data.insert_one(clientdata)  


            